# Préparation des données

In [1]:
import pandas as pd 

import tkinter as tk
from tkinter import filedialog as fd

def select_file_csv():
    
    init = "C:\\Users\\Stanislas Brusselle\\OneDrive\\Documents\\Projet ML sklearn\\des-fichiers-complementaires-3-049-ko\\Telechargement\\data"
    
    file_type = (('csv file', '*.csv'), ('All file', '*.*'))
    file_name = fd.askopenfilename(initialdir = init, title='Fichiers', filetypes = file_type)
    
    return file_name

def select_file_txt():
    
    init = "C:\\Users\\Stanislas Brusselle\\OneDrive\\Documents\\Projet ML sklearn\\des-fichiers-complementaires-3-049-ko\\Telechargement\\data"
    
    file_type = (('csv file', '*.txt'), ('All file', '*.*'))
    file_name = fd.askopenfilename(initialdir = init, title='Fichiers', filetypes = file_type)
    
    return file_name

def cvs_to_dataFrame():
    
    root=tk.Tk()  
    root.overrideredirect(1)

    path=select_file_csv()

    root.destroy()
    
    print("path : " + path)
    
    with open(path, mode='r') as file : 
        fichier = pd.read_csv(file)
        
    return fichier

def txt_to_dataFrame(nom_colonnes, skip_rows):
    
    root=tk.Tk()  
    root.overrideredirect(1)

    path=select_file_txt()

    root.destroy()
    
    print("path : " + path)
    
    with open(path, mode='r') as file : 
        fichier = pd.read_fwf(file, skip_rows=skip_rows, header=None, names=nom_colonnes)

    return fichier

names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

iris_df = cvs_to_dataFrame()
titanic_df = cvs_to_dataFrame()
boston_df = txt_to_dataFrame(names, 22)

path : C:/Users/Stanislas Brusselle/OneDrive/Documents/Projet ML sklearn/des-fichiers-complementaires-3-049-ko/Telechargement/data/iris.csv
path : C:/Users/Stanislas Brusselle/OneDrive/Documents/Projet ML sklearn/des-fichiers-complementaires-3-049-ko/Telechargement/data/titanic_train.csv
path : C:/Users/Stanislas Brusselle/OneDrive/Documents/Projet ML sklearn/des-fichiers-complementaires-3-049-ko/Telechargement/data/boston.txt


# 5) Préparer les catégories

## 5.1) Validation des données 

Comme pour les données numériques, les données catégorielles doivent être préparées. Pour la validation, il peut être utile de vérifier combien d'éléments sont attribués à chaque catégories. 

In [6]:
iris_df['class'].value_counts()

Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: class, dtype: int64

## 5.2) Modification des catégories 

Savoir si les catégories sont ordonnées et si non, les ordonner :

In [9]:
def organize_cat(dataset, column, order):
    if dataset[column].cat.ordered==False:
        dataset[column].cat.reorder_categories(order, ordered=True, inplace=True)
        return dataset
    else : 
        return dataset
    
organize_cat(titanic_df, 'Pclass', [1, 2, 3])

Remplacer une catégorie par une autre : 

In [ ]:
titanic_df['title'].replce('Mlle', 'Miss', inplace=True)

Supprimer les catégories inutiles : 

In [ ]:
titanic_df['title'].cat.remove_unused_categories(inplace=True)

Fusionner plusieurs catégories : 

Ici, je regroupe tous les passagers dont le titre est présent moins de 20 fois en une seule catégorie Other. 
Pour cela, je calcule dans un premier temps le nombre d'occurences, je tri ensuite les catégories peu fréqentes puis les remplace. 

In [ ]:
nb_values = titanic_df['title'].value_counts()
to_replace = nb_values[nb_values<20].index
titanic_df['title']=titanic_df['title'].replace({x:'Other'for x in to_replace}).astype('category')
titanic_df['title'].value_counts()

## 5.3) Quantification 

Cette étape consiste à remplacer une variable catégorielle en une variable numérique. Il n'est pas judicieux de remplacer une catégorie par un nombre. En effet, une catégorie n'a pas de raison de compter plus qu'une autre selon le nombre qui est attribué. On utilisera donc la méthode de dummification. (=> vectoriser les catégories)

In [ ]:
title_df = pd.get_dummies(titanic['title'], prefix = 't')#cela créer une nouvelle colonne title_df qu'il concatener au dataset
new_df = pd.concat([titanic_df, title_df]), axis= 1)

#Je supprime enfin la colonne avec les catégories catégorielles

new_df.drop(columns='title', inplace=True)

La fonction get_dummies a un paramètre drop_first qui précise s'il faut éliminer une colonne (la première). 

# 6) Les données particulières : dates et texte

## 6.1) Préparation des dates

Le format datetime64 de pandas : 
1) joindre chaque partie d'un date dans la même champs (jour, mois et année séparés par des '-')

2) convertir les dates au bon format

Si la date est déjà présente dans le dataset, elle sera reconnue au chargement comme un objet(donc une chaine de caractères), il faudra utiliser pd.to_datetime pour la convertir. Le paramètre dayfirst permet de stipuler si la date est format américain ou européen. 

In [ ]:
#pour réunir tous les caractères de date en un seul champs. 
df['date']= df['jour'].astype('str')+'-'+df['mois'].astype('str')+'-'+df['année'].astype('str')
#pour transformer cette chaine de caractère en format datetime64
df['date']=pd.to_datetime(df['date'], dayfirst=True)

Il y aussi possibilité d'ajouter l'heure au format datetime64 de la même manière que si dessus (les heure et les minutes doivent être séparés par le caractère ':') 

### 6.1.1) Extraire les composants

Une fois la date au bon format il est possible d'extraire les composants de la date. 

df['date']=df['date'].dt.nomcomposant

nomcomposant : 

- dayofyear
- dayofweek
- quarter
- week

### 6.1.2) Gérer les écarts

Il est parfois utile de calculer l'écart entre deux dates. 

La libraire permet d'ajouter ou enlever une durée à un date grâce à la fonction pd.Timedelta. Il suffit d'indiquer en paramètres la durée souhaitée (en millisecondes comme en années) et de l'ajouter ou la soustraire à une variable au bon format (datetime64). 

In [ ]:
df['date'] + pd.Timedelta(days=1)

Il est aussi possible d'ajouter ou soustraire deux dates :

In [ ]:
df['date limite']= df['date actuelle']+df['date rendue']

## 6.2) Préparer les chaines de caractères 

### 6.2.1) Préparer les chaines 

=> la mise de tout texte en minuscule

=> suppression des espaces à gauches ou à droite du texte

=> remplacement de certain caractères comme les accents

Tout passe par la méthode str

In [ ]:
#passage en minuscule : 
titanic_df['Name_lower']=titanic_df['Name'].str.lower
#passage en majuscule
titanic_df['Name_lower']=titanic_df['Name'].str.upper
#supprimer les espaces
titanic_df['Name_lower']=titanic_df['Name'].str.strip
#supprimer les espaces à gauche
titanic_df['Name_lower']=titanic_df['Name'].str.lstrip
#supprimer les espaces à droite
titanic_df['Name_lower']=titanic_df['Name'].str.rstrip
#remplacer un caractère par un autre
#supprimer les espaces à gauchec
titanic_df['Name_lower']=titanic_df['Name'].str.replace(',', '')

### 6.2.2) Effectuer une recherche dans les chaines  

Une fois la chaine de caractères traitée  il est possible d'effectuer une recherche dans celle-ci.

df['caractère']=df['caractère'].str.nomcomposant

nomcomposant : 

- contains : indique si une chaine contient une sous-chaien donnée
- count : compte le nombre d'occurence d'une sous-chaine dans une chaine
- startswitch / endswitch : indique si une chaine commence ou se termine par une sous-chaine
- findall : renvoie toutes les occurences d'une sous-chaine sur tous les enregistrements
- match : indique si les valeurs correspondent à un pattern 
- index : cherche la première occurence d'un sous-chaine et renvoie son index

### 6.2.3) Extraire des sous-chaînes 

Il est également courant de devoir extraire une sous chaine d'un chaine de caractères. De la même manière que pour la recherche, on utilisera le fonction str. 

nomcomposant : 

- slice : extraire une sous-chaine en indiquant son début et sa fin
- extract : extraire une sous-chaine via des expressions régulières (n'extrait que la première correspondance)
- extractall : pareil que la fonction extract mais extrait toutes les correspondances
- split : sépare une chaine de caractères sur un séparateur ou un sous-chaine régulière
- partition : partitionne un chaine de caractères selon un séparateur ou un expression régulière

# 7) Les transfomers : automatiser la préparation 

Une fois le modèle déployé il faudra lui fournir des données en temps réel ou en mode batch. Mais celle-ci nécessiterons aussi un prétraitement car elles devront être au même format que les données d'entrainement. Il faudra créer en parallèle de la préparation des données une pipeline de préparation : la suite de toutes les étapes nécessaires pour passer d'une entrée brute à une donnée traitée. Non seulement il faut conserver les opérations effectuées mais également les valeurs utilisées (telles que min ou max) 

## 7.1) Pipelines avec Scikit-learn 

Je ne détaillerai pas trop cette partie car l'objectif pour moi est de créer mon propre transformer

Il existe une succession d'étapes appelées 'Transformer', chainées sur le dataset pour faire de la gestion des pipelines. 

Création d'un Transformer : 

- init : constructeur du Transformer
- fit : calculer les paramètres nécessaires 
- transform 